# Spectral Extraction Tutorials

In [1]:
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
# from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.table import Table
from astropy.io import fits
from astropy.coordinates import SkyCoord
import warnings
warnings.filterwarnings("ignore")

In [2]:
cat = Table.read('../Data/eROSITA_Catalogues.fit')

In [3]:
cat

IAUName,SkyTile,RA_ICRS,DE_ICRS,GLAT,ELON,FSPSNR
,,deg,deg,deg,deg,
bytes23,int32,float64,float64,float64,float64,int16
1eRASS J080408.5+143949,122075,121.03561512022,14.66374546712,22.582025929909967,120.07636338693729,0
1eRASS J080319.5+144649,122075,120.83127086442,14.78045761483,22.448671900315382,119.85884309038617,0
1eRASS J080415.2+143124,122075,121.06368411779,14.52358196404,22.55054460165602,120.1321124493896,0
1eRASS J080417.9+140221,122075,121.07475186127,14.03929287666,22.36474591720808,120.24316414028134,0
1eRASS J080034.0+142746,122075,120.14208234443,14.46286346619,21.71037055225523,119.26728149190822,0
1eRASS J081101.5+154237,122075,122.75646749818,15.71042539294,24.521942202877863,121.4841879601851,0
1eRASS J081201.5+142011,122075,123.00647208478,14.33642972671,24.198892161183,122.0201413042213,0
1eRASS J080108.9+140515,122075,120.28724719578,14.08764151465,21.686968726871388,119.48185685497229,0


In [4]:
is_spurious = cat['FSPSNR'].astype(bool)
cat = cat[~is_spurious]

cat_coord = SkyCoord(cat['RA_ICRS'], cat['DE_ICRS'], unit='deg')

coord_obj = SkyCoord(83.63240, 22.01740, unit='deg')

distance = cat_coord.separation(coord_obj)

In [5]:
Image_file = '../Data/Images/merged_exp_corr_200_2300.fits'

In [6]:
from astropy.wcs import WCS

# Read the image file header
with fits.open(Image_file) as hdul:
    header = hdul[0].header

# Extract the WCS information from the header
wcs = WCS(header)

# Calculate the extent of the image
naxis1 = header['NAXIS1']
naxis2 = header['NAXIS2']
ra_min, dec_min = wcs.wcs_pix2world([[0, 0]], 0)[0]
ra_max, dec_max = wcs.wcs_pix2world([[naxis1, naxis2]], 0)[0]

# Define the extent of the image
extent = SkyCoord([ra_min, ra_max], [dec_min, dec_max], unit='deg')
extent

<SkyCoord (ICRS): (ra, dec) in deg
    [(85.55021296, 20.20636293), (81.66400501, 23.80665032)]>

In [7]:
# Apply cuts on the catalog using ra_min, ra_max, dec_min, dec_max
cat_cut = cat[(cat['RA_ICRS'] >= ra_max) & (cat['RA_ICRS'] <= ra_min) & 
              (cat['DE_ICRS'] >= dec_min) & (cat['DE_ICRS'] <= dec_max)]

with open('../Data/Spectrum/cat_deg.reg', 'w') as f:
    # f.write('fk5\n')
    for row in cat_cut:
        f.write(f'circle({row["RA_ICRS"]},{row["DE_ICRS"]},50") # color=red\n')

In [8]:
cat_src = cat[distance < 500/3600 * u.deg]

with open('../Data/Spectrum/src_deg.reg', 'w') as f:
    # f.write('fk5\n')
    f.write(f'circle({coord_obj.ra.deg},{coord_obj.dec.deg},500") # color=white\n')
    for row in cat_src:
        f.write(f'-circle({row["RA_ICRS"]},{row["DE_ICRS"]},50") # color=red\n')

In [9]:
cat_bg = cat[(distance >= 1000/3600 * u.deg) & (distance < 2500/3600 * u.deg)]

with open('../Data/Spectrum/bg_deg.reg', 'w') as f:
    # f.write('fk5\n')
    f.write(f'annulus({coord_obj.ra.deg},{coord_obj.dec.deg},1000", 2500") # color=green\n')
    for row in cat_bg:
        f.write(f'-circle({row["RA_ICRS"]},{row["DE_ICRS"]},50") # color=red\n')

## Generate spectra, ARFs, and RMFs

In [32]:
def run_srctool(eventfiles, srccoord, prefix, srcreg, backreg, tstep=0.5, xgrid=1, log_file=None):
    subprocess.run(['srctool',
                    f'eventfiles={eventfiles}',
                    f'srccoord={srccoord}',
                    f'prefix={prefix}',
                    'todo=SPEC ARF RMF',
                    'insts=1 2 3 4 6',  # read from TM12346 only
                    'writeinsts=8',  # write spectra of individual TMs as well as TM8
                    f'srcreg={srcreg}',
                    f'suffix=_ts{tstep}_xg{xgrid}.fits',
                    f'backreg={backreg}',  # we don't need a spectrum from a background region as we are interested in the astrophysical background ourselves, not sources on top of the background
                    'exttype=TOPHAT',  # assume the source is uniformly bright to estimate the effective area
                    'extpars=15500',  # approximate diameter of the tophat circle that will cover the full spectral extraction region (in arcsec)
                    f'tstep={tstep}',  # time step for the light curve
                    f'xgrid={xgrid}',  # these numbers are the trade-off between accuracy and time.
                    'psftype=NONE',  # do not correct for psf
                    'pat_sel=15',  # all valid patterns
                    'clobber=yes'
                    ],
                   stdout=log_file,
                   stderr=log_file)

In [33]:
with open('../Data/Spectrum/Spec_SRC_BG_log.txt', 'w+') as log_file:

    run_srctool('../Data/Filtered_data/Merged/Merged_020_s05_TM0_Events.fits', 
                'icrs; 83.63240,+22.01740',
                '../Data/Spectrum/Spec_Src_', 
                '../Data/Spectrum/src_deg.reg', 
                '../Data/Spectrum/bg_deg.reg', log_file=log_file)
    
    log_file.seek(0)
    log_content = log_file.readlines()
    srctool_count = sum(1 for line in log_content if 'srctool: DONE' in line)
    if srctool_count == 1:
        print('srctool tasks completed successfully')

srctool tasks completed successfully
